# Extracting Data from Foursquare
06/5/19

* Refactor code from "Exploring Foursquare"

In [1]:
import numpy as np # library to handle data in a vectorized manner
# import math   # for ceil function

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# #!conda install -c conda-forge geopy --yes # uncomment this line if needed
# from geopy.geocoders import Nominatim # convert address into latitude and longitude

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
# import matplotlib.cm as cm
# import matplotlib.colors as colors

# import k-means from clustering stage
# from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if needed
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
# @hidden cell
# Foursquare information

CLIENT_ID = 'CWCPNAVXDH3TI1BGS4VED4ANSUKEFGHBA4511GRPYPKPNJRD' # your Foursquare ID
CLIENT_SECRET = 'ALMFEYIENSPH3RV3TQB1NGWTKTANVJ5QTHAVZ5B1GRJWIP21' # your Foursquare Secret

# Foursquare query information that will be constant during this project
VERSION = '20190214' # Foursquare API version
LIMIT = 120 

In [0]:
# define function that extracts the category of the venue
# This function comes from Coursera Applied Data Science Capstone class
# used in Foursquare section

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
          
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
def split_box(box, purpose="number"):
    # Breaks a map "box" into smaller pieces
    # purpose = "size" - if the size of the box exceeds Foursquare's max size
    # purpose = "number" - if the number of returned results is 100 or more,
    #                      indicating that the query "maxed out"
    
    MAX_DELTA_LAT = 0.9   # Max allowable difference in latitude between corners
    MAX_DELTA_LNG = 0.9   # Max difference in longitude between corners
    
    delta_lat = abs(box[0][0] - box[1][0])
    delta_lng = abs(box[0][1] - box[1][1])
    
    boxes = [] # Will hold 1 or more map boxes
    
    if purpose == "number":  # Splitting box because too many results returned
        lat_divisions = lng_divisions = 2 # Break the box into 4 sub-boxesx
        
    elif purpose == "size":  # Splitting because box is too large for Foursquare
        lat_divisions = int(np.ceil(delta_lat / MAX_DELTA_LAT))
        lng_divisions = int(np.ceil(delta_lng / MAX_DELTA_LNG))
        
    else:  
        raise ValueError("This function only supports 'size' and 'number' box splitting")
    
    # Create sub-boxes based on number of lat & lng divisions
    for i in range(lat_divisions):
        for j in range(lng_divisions):
            lat_side = delta_lat / lat_divisions
            lng_side = delta_lng / lng_divisions
            swij = (box[0][0] + i * lat_side, box[0][1] + j * lng_side)
            neij = (swij[0] + lat_side, swij[1] + lng_side)
            boxes.append((swij, neij))            
    
    return boxes   

In [0]:
def query_box(box, codes):
    print(".", end="") # Indicate activity while performing queries

  # define the Foursquare URL

  # Foursquare category ID requires comma-separated text string
    code_string = ",".join(codes)    
    
    url = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}' + \
        f'&client_secret={CLIENT_SECRET}&v={VERSION}&sw={box[0][0]},{box[0][1]}' + \
        f'&ne={box[1][0]},{box[1][1]}' + \
        f'&categoryId={code_string}&limit={LIMIT}'
  
  # submit the url and capture the returned text
    venues = requests.get(url).json()["response"]['groups'][0]['items']
  
  # if any venues are captured, process and return dataframe. Otherwise, return None. 
    if venues:
        venues = json_normalize(venues) # flatten JSON

        # select desired columns
        filtered_columns = ['venue.name', 'venue.categories', # 'venue.location.postalCode', 
                        'venue.location.lat', 'venue.location.lng', 'venue.location.state']
        venues = venues.loc[:, filtered_columns]

        # filter the category for each row
        venues['venue.categories'] = venues.apply(get_category_type, axis=1)

        # clean column names
        venues.columns = [col.split(".")[-1] for col in venues.columns]
  
        return venues
  
    else:
        return None  # no venues in box meet criteria    

In [0]:
def shrink_box(box, codes, venue_list, query_list):
    
    # Get venues for box (if any) in Pandas dataframe
    box_results = query_box(box, codes)

    try: # This will succeed if 1 or more venues are returned in box_results
        num_clients = box_results.shape[0]

        if num_clients < 100:  
            # Case where results did not "max out" at 100 returned venues
            venue_list += box_results.values.tolist()
            query_list.append((box, num_clients))

        else:  # This is the case where num_clients > 100
            for sub_box in split_box(box, purpose="number"):
                shrink_box(sub_box, codes, venue_list, query_list)

    except: # happens if query returns no results
        query_list.append((box, 0))

    return # venue_list and query_list are mutable, no need to return

In [0]:
def GetVenuesByBox(start_box,  # a tuple containing sw & ne corners of box
                   codes):     # a list of Foursquare category strings

    venue_list = []      # list of venus of interest
    query_list = []      # boxes and number of venues in each, for visualization
    
    # if the start box is too large, split it into smaller boxes
    for box in split_box(start_box, purpose="size"):
        # List will contain one or more boxes. Obtain venues for each
        shrink_box(box, codes, venue_list, query_list)
        
    # Add venue data to a dataframe    
    venues = pd.DataFrame(venue_list, columns=['name', 'categories', # 'postalCode', \
                                                  'lat', 'lng', 'state'])
    return venues, query_list

# Requesting Data from Foursquare

Foursquare allows you to request data by defining a bounding box using latitude & longitude coordinates. I want to request all Asian (including Indian & Pakistani) restaurants in a box drawn around the state of Georgia. 

There are two problems. First, Foursquare has a size limit (about 10,000 square kilometers). Georgia is much larger. Also, Foursquare will only return 100 venues per request (at least with my account type). My code handles these issues by dividing the box into smaller sub-boxes until all boxes are small enough for Foursquare. Then every time a query on a sub-box returns 100 results, the sub-box is split into 4 pieces and a new query is run on each one. This process repeats until all queries return <100 results. Any queries with 100 results are thrown away to avoid duplication.

Let's begin by drawing a large box around Georgia and requesting data. We will cover parts of the adjacent states but we will drop any rows that aren't in Georgia later.

In [41]:
# Define the southwest and northeast corners of a box around the state of Georgia

# fsw = (33.946964, -84.150187)  # Small Box for testin
# fne = (33.971041, -84.121381)
fsw = (30.357851, -85.605165)  # Southwest corner, Georgia USA
fne = (35.000659, -80.839729)  # Northeast corner

# The following venue categories come from Foursquare. Note that each 
# category has sub-categories (for example, "Korean Restaurant" in "Asian").
restaurant_codes =  [#'4bf58dd8d48988d142941735',   # Asian Restaurant
                     '4bf58dd8d48988d10f941735',   # Indian Restaurant
                     '52e81612bcbc57f1066b79f8',    # Pakistani Restaurant
                     #'52af3b773cf9994f4e043c03'    # Szechuan Restaurant
                    ]   

# Request the data
df_venues, query_list = GetVenuesByBox((fsw,fne), restaurant_codes)

............................................

In [44]:
df_venues.describe(include='all')

,name,categories,lat,lng,state
count,384,382,384.000000,384.000000,384
unique,355,44,NaN,NaN,8
top,Chinese Dhaba,Indian Restaurant,NaN,NaN,GA
freq,4,301,NaN,NaN,313
mean,NaN,NaN,33.706412,-83.832057,NaN
std,NaN,NaN,0.899987,1.081389,NaN
min,NaN,NaN,30.427154,-85.550739,NaN
25%,NaN,NaN,33.784594,-84.340113,NaN
50%,NaN,NaN,33.910262,-84.237521,NaN
75%,NaN,NaN,34.048314,-84.106039,NaN


## Data gathering process

Let's see how the code split the data request. Each box in the picture below represents a Foursquare query that returned 99 or fewer results. The shade of the box shows the number of returned values (you can click on the box to see the number). This is NOT a picture of the data -- we will see that in much more detail later.

In [45]:
# Center the map on the middle of the bounding box defined above 
center = [(fsw[0]+fne[0])/2, (fsw[1]+fne[1])/2]

# Find the maximum number of venues in any square (to calculate fill opacity)
max_clients = max([query[1] for query in query_list])

# Visualize all queries on a map, with fill color indicating number of restaurants
map_georgia = folium.Map(location=center, zoom_start=7, control_scale=True)   
    
for box in query_list:
    sw, ne = box[0][0], box[0][1]
    opacity = box[1]/max_clients   # Clients in this box / max across all boxes          
    folium.Rectangle([sw, ne], 
                     popup=str(box[1]), color="black", opacity=0.99, fill=True, 
                     fill_color='orange', weight = 1, 
                     fill_opacity=(opacity)).add_to(map_georgia)
    
map_georgia

In the picture above you can see we started with a 6x6 grid, for 36 queries, to satisfy the size limit. After throwing away all queries returning 100 results (the max limit), how many successful queries did we end up with?

In [46]:
# Number of queries returning <100 results
len(query_list)

42

To get to a total of 141, boxes were split (141 - 36)/3 = 35 times. Data from those boxes were  thrown away, meaning 35 queries were wasted. The total number of queries to get my data from Georgia was 141 + 35 = 176. 

However, the smallest boxes are about 2 square miles in size. Georgia covers almost 60,000 square miles. **Over 25,000 queries would be needed to get this data with equal-sized boxes.** So my method is a bit better than that.




# Data Cleaning 

Now let's look at the data we've obtained, and get rid of any venues we don't need for our analysis

In [47]:
print("Number of restaurants in data set: ",df_venues.shape[0],"\n\n")
df_venues.head()

Number of restaurants in data set:  384 




,name,categories,lat,lng,state
0,Waffle House,Breakfast Spot,30.725953,-85.185990,FL
1,Mayuri Indian Restaurant,Indian Restaurant,30.434194,-84.262330,FL
2,Persis Grill,Indian Restaurant,30.460266,-84.224231,FL
3,Essence of India,Indian Restaurant,30.436881,-84.265688,FL
4,Samrat Indian Restaurant,Indian Restaurant,30.427154,-84.237288,FL


My bounding-box approach obtained restaurants in parts of adjacent states. Let's get rid of those.

In [48]:
# Which states are included in the data set?

print(list(set(df_venues.state)))

['SC', 'South Carolina', 'Alabama', 'AL', 'GA', 'Georgia', 'FL', 'Florida']


In [49]:
# Get rid of restaurants in other states and count the results.
df_georgia = df_venues[df_venues['state'].isin(['GA', 'Georgia'])]
print("Number of restaurants Georgia: ",df_georgia.shape[0],"\n\n")
df_georgia.head()


Number of restaurants Georgia:  322 




,name,categories,lat,lng,state
11,Jazz Table,Indian Restaurant,30.835180,-83.325075,GA
12,Passage 2India,Indian Restaurant,30.871033,-83.289801,GA
13,Tandoor,Indian Restaurant,30.844594,-83.332547,GA
14,Blakes Kitchen Table,Indian Restaurant,30.919344,-83.361744,GA
15,Taste of India,Indian Restaurant,30.866921,-83.295485,Georgia


In [50]:
df_georgia.describe(include='all')

,name,categories,lat,lng,state
count,322,322,322.000000,322.000000,322
unique,296,26,NaN,NaN,2
top,Chinese Dhaba,Indian Restaurant,NaN,NaN,GA
freq,4,265,NaN,NaN,313
mean,NaN,NaN,33.749211,-84.111439,NaN
std,NaN,NaN,0.601438,0.640293,NaN
min,NaN,NaN,30.835180,-85.258985,NaN
25%,NaN,NaN,33.796777,-84.349300,NaN
50%,NaN,NaN,33.903164,-84.271429,NaN
75%,NaN,NaN,34.035847,-84.164537,NaN


That's better. In theory, this approach would prevent duplication in the data set, but I always like to be sure. Let's check for duplicate venues.

In [15]:
print("Before dropping duplicates: ",df_georgia.shape[0])
df_georgia = df_georgia.drop_duplicates(keep='first')
print("After dropping duplicates: ",df_georgia.shape[0])

Before dropping duplicates:  3375
After dropping duplicates:  3375


In [36]:
df_georgia[df_georgia['postalCode'].isnull()].shape[0]

368

In [34]:
df_georgia.describe(include='all')

,name,categories,postalCode,lat,lng,state
count,3375,3375,3007,3375.000000,3375.000000,3375
unique,2602,75,327,NaN,NaN,2
top,Panda Express,Chinese Restaurant,30096,NaN,NaN,GA
freq,55,1040,228,NaN,NaN,3286
mean,NaN,NaN,NaN,33.545832,-83.919630,NaN
std,NaN,NaN,NaN,0.800516,0.955133,NaN
min,NaN,NaN,NaN,30.681651,-85.512472,NaN
25%,NaN,NaN,NaN,33.479799,-84.386459,NaN
50%,NaN,NaN,NaN,33.879370,-84.215070,NaN
75%,NaN,NaN,NaN,33.988102,-83.968619,NaN


Foursquare did a good job of giving us a unique data set. I also looked at the data and confirmed that all rows contain values for restaurant name, categories, lat, and lng. Now let's look at the restaurant categories. Did we only get the ones we wanted?

In [0]:
# sorted(list(set(df_georgia['categories'])))
set(df_georgia['categories'])

{'American Restaurant',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Bar',
 'Breakfast Spot',
 'Bubble Tea Shop',
 'Buffet',
 'Café',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Coffee Shop',
 'Deli / Bodega',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Food Court',
 'Food Truck',
 'Fried Chicken Joint',
 'Garden',
 'Gas Station',
 'Grocery Store',
 'Hot Dog Joint',
 'Hotel Bar',
 'Hotpot Restaurant',
 'Indian Chinese Restaurant',
 'Indian Restaurant',
 'Indonesian Restaurant',
 'Japanese Curry Restaurant',
 'Japanese Restaurant',
 'Jiangsu Restaurant',
 'Karaoke Bar',
 'Korean Restaurant',
 'Malay Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Mongolian Restaurant',
 'Noodle House',
 'Poke Place',
 'Ramen Restaurant',
 'Restaurant',
 'Salon / Barbershop',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shabu-Shabu Restaurant',
 'Shanghai Restaurant',
 'Soup Place',

Convenience store? Italian restaurant? Garden? American restaurant? Clearly we need to get rid of some of these venues. In the cell below, I go through the list of categories and define a list of ones we want to get rid of. In many cases, I used a command like the following to see which restaurants were in a suspicious category: 

     df_georgia[df_georgia['categories'].isin(['Taco Place'])]

You can see my comments in the lines below.

In [0]:
bad_list = [ 'American Restaurant',
 # 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Bar',
 'Breakfast Spot',
 # 'Bubble Tea Shop',
 # 'Buffet',
 'Café',
 # 'Cantonese Restaurant',
 'Caribbean Restaurant',
 # 'Chaat Place',
 # 'Chinese Restaurant',
 'Cocktail Bar',
 'Coffee Shop',
 'Convenience Store',
 'Deli / Bodega',
 # 'Dim Sum Restaurant',
 # 'Dumpling Restaurant',
 'Farmers Market',
 'Fast Food Restaurant',
 # 'Filipino Restaurant',
 # 'Food & Drink Shop',  # One instance. Indian.
 'Food Court',
 # 'Food Truck',
 # 'Fried Chicken Joint',
 # 'Garden',   # One instance. Chinese restaurant.
 'Gas Station',
 'Grocery Store',
 # 'Hot Dog Joint',  # One instance, Korean fried hotdogs. Must. Try. This. 
 'Hotel Bar',
 # 'Hotpot Restaurant',
 # 'Hunan Restaurant',
 # 'Indian Chinese Restaurant',
 # 'Indian Restaurant',
 # 'Indian Sweet Shop',
 # 'Indonesian Restaurant',
 'Italian Restaurant',
 # 'Japanese Curry Restaurant',
 # 'Japanese Restaurant',
 # 'Jiangsu Restaurant',  # One instance, Indian
 'Juice Bar',
 # 'Karaoke Bar',
 # 'Korean Restaurant',
 # 'Malay Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 # 'Mongolian Restaurant',
 'New American Restaurant',
 # 'Noodle House',
 # 'North Indian Restaurant',
 'Pizza Place',
 # 'Poke Place',
 # 'Ramen Restaurant',
 # 'Restaurant',  # 8 venues, 3 asian 
 'Salon / Barbershop',
 # 'Sandwich Place',  # Two locations, one of which is Korean
 'Seafood Restaurant',
 # 'Shabu-Shabu Restaurant',
 # 'Shanghai Restaurant',
 'Snack Place',
 'Soup Place',  # One instance. It is Asian, but out of business
 # 'South Indian Restaurant',
 'Sports Bar',
 # 'Steakhouse',  # Two places, both Japanese
 'Supermarket',
 # 'Sushi Restaurant', #110 locations, not all Japanese
 # 'Szechuan Restaurant',
 # 'Taco Place',  # One location "Hankook Taqueria" Korean
 # 'Taiwanese Restaurant',
 # 'Tea Room', # One Location, Chinese
 # 'Thai Restaurant',
 # 'Vegetarian / Vegan Restaurant', # Two locations, unspecified Asian & Chinese
 # 'Vietnamese Restaurant',
 'Wings Joint']

Let's see how many restaurants are in the categories we don't want, and drop them from the data frame.

In [0]:
print("Number of restaurants in unwanted categories (dropped): ", df_georgia[df_georgia['categories'].isin(bad_list)].shape[0])
df_georgia = df_georgia[~df_georgia['categories'].isin(bad_list)]
print("Restaurants remaining in dataset: ", df_georgia.shape[0])
df_georgia.head()

Number of restaurants in unwanted categories (dropped):  107
Restaurants remaining in dataset:  3268


,name,categories,postalCode,lat,lng,state
114,Yuki Express,Asian Restaurant,39819,30.903404,-84.537571,GA
115,Makan,Asian Restaurant,30030,30.903800,-84.575470,GA
116,Star China Buffet,Chinese Restaurant,39819,30.902765,-84.554162,GA
117,Yuki Express Japanese Restaurant,Japanese Restaurant,39819,30.903800,-84.575470,GA
118,Jin's Chinese Buffet,Chinese Restaurant,39828,30.891642,-84.207543,GA


In [0]:
df_georgia[df_georgia['categories'].isin(['Chinese Restaurant'])].shape

(1044, 6)

In [0]:
sorted(list(set(df_georgia['categories'])))

['Asian Restaurant',
 'Bubble Tea Shop',
 'Buffet',
 'Cantonese Restaurant',
 'Chinese Restaurant',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Filipino Restaurant',
 'Food Truck',
 'Fried Chicken Joint',
 'Garden',
 'Hot Dog Joint',
 'Hotpot Restaurant',
 'Indian Chinese Restaurant',
 'Indian Restaurant',
 'Indonesian Restaurant',
 'Japanese Curry Restaurant',
 'Japanese Restaurant',
 'Jiangsu Restaurant',
 'Karaoke Bar',
 'Korean Restaurant',
 'Malay Restaurant',
 'Mongolian Restaurant',
 'Noodle House',
 'Poke Place',
 'Ramen Restaurant',
 'Restaurant',
 'Sandwich Place',
 'Shabu-Shabu Restaurant',
 'Shanghai Restaurant',
 'Steakhouse',
 'Sushi Restaurant',
 'Szechuan Restaurant',
 'Taco Place',
 'Taiwanese Restaurant',
 'Tea Room',
 'Thai Restaurant',
 'Vietnamese Restaurant']

In [0]:
df_georgia[df_georgia['categories'].isin(['Korean Restaurant'])].groupby('name').count().sort_values(by='state',ascending=False)

In [0]:
df_georgia[df_georgia['categories'].isin(bad_list)].shape

(107, 6)

In [0]:
df_georgia[df_georgia['categories'].isin(['Food'])]

,name,categories,postalCode,lat,lng,state
679,Bamboo Garden,Garden,31419,31.981504,-81.166772,GA


In [0]:
print(len(set(df_georgia['categories'])))
good_list = set(df_georgia['categories']) - set(bad_list)
good_list

76


{'Asian Restaurant',
 'Bubble Tea Shop',
 'Buffet',
 'Cantonese Restaurant',
 'Chaat Place',
 'Chinese Restaurant',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Filipino Restaurant',
 'Food & Drink Shop',
 'Food Truck',
 'Fried Chicken Joint',
 'Garden',
 'Hot Dog Joint',
 'Hotpot Restaurant',
 'Indian Chinese Restaurant',
 'Indian Restaurant',
 'Indian Sweet Shop',
 'Indonesian Restaurant',
 'Japanese Curry Restaurant',
 'Japanese Restaurant',
 'Jiangsu Restaurant',
 'Karaoke Bar',
 'Korean Restaurant',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mongolian Restaurant',
 'Noodle House',
 'North Indian Restaurant',
 'Pakistani Restaurant',
 'Poke Place',
 'Ramen Restaurant',
 'Restaurant',
 'Sandwich Place',
 'Shabu-Shabu Restaurant',
 'Shanghai Restaurant',
 'South Indian Restaurant',
 'Steakhouse',
 'Sushi Restaurant',
 'Szechuan Restaurant',
 'Taco Place',
 'Taiwanese Restaurant',
 'Tea Room',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant'}

In [0]:
len(query_list)

141

In [0]:
print(df_venues.shape)
df_venues.head()


(374, 6)


,name,categories,postalCode,lat,lng,state
0,Ashiana,Indian Restaurant,30071,33.914464,-84.207432,GA
1,I Luv Pho 2,Asian Restaurant,30096,33.942934,-84.159522,GA
2,Seo Ra Beol Restaurant,Asian Restaurant,30096,33.963876,-84.141964,GA
3,SunO Dessert,Asian Restaurant,30096,33.969180,-84.145040,GA
4,Chow King,Asian Restaurant,NaN,33.932097,-84.179774,GA


In [0]:
query_list

[(((33.913157, -84.22063), (33.969297, -84.141344)), 66),
 (((33.913157, -84.141344), (33.941227, -84.101701)), 2),
 (((33.913157, -84.101701), (33.941227, -84.06205800000001)), 3),
 (((33.941227, -84.141344), (33.955262, -84.1215225)), 31),
 (((33.941227, -84.1215225), (33.955262, -84.10170099999999)), 2),
 (((33.955262, -84.141344), (33.962279499999994, -84.13143325)), 51),
 (((33.955262, -84.13143325), (33.962279499999994, -84.1215225)), 17),
 (((33.962279499999994, -84.141344), (33.969297, -84.13143325)), 31),
 (((33.962279499999994, -84.13143325), (33.969297, -84.1215225)), 4),
 (((33.955262, -84.1215225), (33.969297, -84.10170099999999)), 7),
 (((33.941227, -84.101701), (33.969297, -84.06205800000001)), 2),
 (((33.969297, -84.22063), (34.025437, -84.141344)), 97),
 (((33.969297, -84.141344), (34.025437, -84.06205800000001)), 61)]

In [0]:
a_dictionary = {'Indonesian Restaurant': 'a',
 'Poke Place': 'b',
 'Noodle House': 'c',
 'Ramen Restaurant': 'd',
 'Bubble Tea Shop': 'e',
 'Indian Restaurant': 'f',
 'Steakhouse': 'g'}

In [0]:
a_dictionary['Noodle House']

'c'

In [0]:
df_georgia[['categories','name']].groupby('categories').count()\
.sort_values(by='name', ascending=False)

,name
categories,
Chinese Restaurant,1040
Asian Restaurant,654
Japanese Restaurant,449
Indian Restaurant,250
Sushi Restaurant,243
Korean Restaurant,211
Thai Restaurant,176
Vietnamese Restaurant,92
Noodle House,32


# BLANK LINES FOR TESTING

In [51]:
df_georgia.describe(include='all')

,name,categories,lat,lng,state
count,322,322,322.000000,322.000000,322
unique,296,26,NaN,NaN,2
top,Chinese Dhaba,Indian Restaurant,NaN,NaN,GA
freq,4,265,NaN,NaN,313
mean,NaN,NaN,33.749211,-84.111439,NaN
std,NaN,NaN,0.601438,0.640293,NaN
min,NaN,NaN,30.835180,-85.258985,NaN
25%,NaN,NaN,33.796777,-84.349300,NaN
50%,NaN,NaN,33.903164,-84.271429,NaN
75%,NaN,NaN,34.035847,-84.164537,NaN


In [53]:
set(df_georgia['categories'])

{'Arcade',
 'Bar',
 'Big Box Store',
 'Breakfast Spot',
 'Brewery',
 'Chaat Place',
 'Chinese Restaurant',
 'Fast Food Restaurant',
 'Food & Drink Shop',
 'Food Truck',
 'Gourmet Shop',
 'Grocery Store',
 'Hotel',
 'Indian Chinese Restaurant',
 'Indian Restaurant',
 'Indian Sweet Shop',
 'Italian Restaurant',
 'Mediterranean Restaurant',
 'North Indian Restaurant',
 'Pakistani Restaurant',
 'Pizza Place',
 'Sandwich Place',
 'South Indian Restaurant',
 'Sports Bar',
 'Tea Room',
 'Vegetarian / Vegan Restaurant'}

In [58]:
df_georgia[df_georgia['categories'].isin(['Grocery Store'])]

,name,categories,lat,lng,state
56,India Stop Shop,Grocery Store,33.489505,-84.583528,GA
74,Patel Brothers Grocery,Grocery Store,33.797515,-84.281206,GA
122,Whole Foods Market,Grocery Store,33.775056,-84.366082,GA
126,EZ Mart,Grocery Store,33.797039,-84.407690,GA
169,Trader Joe's,Grocery Store,33.926248,-84.378479,GA
170,Publix,Grocery Store,33.851035,-84.359765,GA
171,Whole Foods Market,Grocery Store,33.840813,-84.381535,GA
295,Spices Hut,Grocery Store,34.060564,-84.237753,GA
302,Cherians,Grocery Store,34.105463,-84.176128,GA
304,Suvidha Indo-Pak Grocery,Grocery Store,34.077406,-84.162503,GA


In [63]:
df_georgia.groupby('categories').count().sort_values(by=['name'], ascending=False).tail(19)

,name,lat,lng,state
categories,,,,
Big Box Store,2,2,2,2
Food Truck,2,2,2,2
Italian Restaurant,1,1,1,1
Tea Room,1,1,1,1
Sports Bar,1,1,1,1
Sandwich Place,1,1,1,1
Pizza Place,1,1,1,1
Mediterranean Restaurant,1,1,1,1
Arcade,1,1,1,1
